In [2]:
import tkinter as tk
from tkinter import ttk
import mysql.connector as connector

def connect_to_database():
    global connection, cursor
    try:
        # Retrieve the password from the entry widget
        password = password_entry.get()

        # Establish connection between Python and MySQL database via connector API
        connection = connector.connect(
            host="localhost",
            user="DisenchantmentAdmin",  # use your own
            password=password,  # use the entered password
            database="testdisenchantment",  # set the default database
            raise_on_warnings=True
        )

        print("Connection between MySQL and Python is established.\n")
        connection_label.config(text="Connected")
        open_second_window()
    except connector.Error as err:
        print(f"Error connecting to the database: {err}")
def quick_connect():
    try:
        # Retrieve the password from the entry widget
        password = password_entry.get()

        # Establish connection between Python and MySQL database via connector API
        connection = connector.connect(
            host="localhost",
            user="DisenchantmentAdmin",  # use your own
            password=password,  # use the entered password
            database="testdisenchantment",  # set the default database
            raise_on_warnings=True
        )
        cursor = connection.cursor()
        return connection, cursor
        print("Connection between MySQL and Python is established.\n")
    except connector.Error as err:
        print(f"Error connecting to the database: {err}")
    # Create a cursor object to execute SQL statements

def quick_close():
    try:
        connection.close()
    except connector.Error as err:
        print(f"Error closing connection to the database: {err}")
    # Create a cursor object to execute SQL statements

def close_connection():
    global connection
    try:
        # Close the database connection
        connection.close()
        print("Connection to the database is closed.\n")
        connection_label.config(text="Not Connected")
    except connector.Error as err:
        print(f"Error closing the connection: {err}")

#----------------------------------------------------------------------------------------------------------------------------
#----------------------------------------------------------------------------------------------------------------------------

def open_second_window():
    second_window = tk.Toplevel()
    second_window.title("Second Window")

    # Create the "Elements" button in the second window
    elements_button = tk.Button(second_window, text="Elements", command=open_admin_elements_window)
    elements_button.pack()

    # Create the "Episodes" button in the second window
    episodes_button = tk.Button(second_window, text="Episodes", command=open_admin_episodes_window)
    episodes_button.pack()

    # Create the "Traits" button in the second window
    traits_button = tk.Button(second_window, text="Traits", command=open_admin_traits_window)
    traits_button.pack()

#-------------------------------------------------------------------------------------------------------------------------------
#-------------------------------------------------------------------------------------------------------------------------------

def add_episode(tree, part_id_entry, episode_name_entry, synopsis_entry):
    # Get the values from text boxes
    episode_name = episode_name_entry.get()
    part_id = part_id_entry.get()
    synopsis = synopsis_entry.get()

    try:
        # Establish connection between Python and MySQL database via connector API
        connection = connector.connect(
            host="localhost",
            user="DisenchantmentAdmin",  # use your own
            password=password_entry.get(),  # use the entered password
            database="testdisenchantment",  # set the default schema
            raise_on_warnings=True
        )

        # Create a cursor object to execute SQL statements
        cursor = connection.cursor()

        # Insert the episode into the 'episode' table
        insert_query = "INSERT INTO episode (PartID, EpisodeName, Synopsis) VALUES (%s, %s, %s)"
        cursor.execute(insert_query, (part_id, episode_name, synopsis))
        connection.commit()
        part_id_entry.delete(0, tk.END)
        episode_name_entry.delete(0, tk.END)
        synopsis_entry.delete(0, tk.END)
        print("Episode added successfully.")

        # Fetch data from the updated 'episode' table
        cursor.execute("SELECT EpisodeID, EpisodeName, PartID, Synopsis FROM episode")

        # Clear the tree view
        tree.delete(*tree.get_children())

        # Insert the data into the tree view
        for row in cursor:
            tree.insert("", tk.END, values=row)

    except connector.Error as err:
        print(f"Failed to add episode: {err}")

    finally:
        # Close the cursor and the connection
        cursor.close()
        connection.close()
##-----------------------------------------------------------------------------------------------------------------------
##------------------------------------------------------------------------------------------------------------------------

def add_trait(tree, trait_name_entry, trait_universal_description_entry):
    # Get the values from text boxes
    trait_name = trait_name_entry.get()
    print("trait name")
    trait_universal_description = trait_universal_description_entry.get()
    
    try:
        connection, cursor = quick_connect()

        # Call the AddTrait procedure
        try:
            cursor.callproc("AddTrait", (trait_name, trait_universal_description))
            connection.commit()
        except connector.Error as error:
            print(f"Error adding trait: {error}")

        # Fetch data from the updated 'trait' table
        cursor.execute("SELECT TraitID, TraitName, TraitUniversalDescription FROM trait")

        # Clear the tree view
        tree.delete(*tree.get_children())

        # Insert the data into the tree view
        for row in cursor:
            tree.insert("", tk.END, values=row)

    except connector.Error as err:
        print(f"Failed to add trait: {err}")

    finally:
        # Close the cursor and the connection
        cursor.close()
        connection.close()
##-----------------------------------------------------------------------------------------------------------------------
##------------------------------------------------------------------------------------------------------------------------

def open_admin_episodes_window():
    admin_episodes_window = tk.Toplevel(window)
    admin_episodes_window.title("Admin Episodes")

    # Create the tree view
    tree = ttk.Treeview(admin_episodes_window, columns=("EpisodeID", "EpisodeName", "PartID", "Synopsis"))
    tree.heading("EpisodeID", text="Episode ID")
    tree.heading("PartID", text="Part ID")
    tree.heading("EpisodeName", text="Episode Name")
    tree.heading("Synopsis", text="Synopsis")
    tree.pack()

    # Fetch data from the 'episodes' table
    try:
        # Establish connection between Python and MySQL database via connector API
        connection = connector.connect(
            host="localhost",
            user="DisenchantmentAdmin",  # use your own
            password=password_entry.get(),  # use the entered password
            database="testdisenchantment",  # set the default schema
            raise_on_warnings=True
        )

        # Create a cursor object to execute SQL statements
        cursor = connection.cursor()

        # Fetch data from the 'episodes' table
        cursor.execute("SELECT EpisodeID, EpisodeName, PartID, EpisodeName, Synopsis FROM episode")

        # Insert the data into the tree view
        for row in cursor:
            tree.insert("", tk.END, values=row)

    except connector.Error as err:
        print(f"Error fetching data from the 'episodes' table: {err}")

    finally:
        # Close the cursor and the connection
        cursor.close()
        connection.close()
    # Create the "Scenes" button in the episodes window
    scenes_button = tk.Button(admin_episodes_window, text="Scenes", command=lambda: open_admin_scenes_window(tree.set(tree.selection(), "#1")))
    scenes_button.pack(pady=10)
    scenes_button.pack(pady = 10)
    # Create frames for interactive text boxes and labels
    text_boxes_frame = tk.Frame(admin_episodes_window)
    text_boxes_frame.pack(pady=20)

    # Create the labels for text boxes
    part_id_label = tk.Label(text_boxes_frame, text="Part ID:")
    part_id_label.grid(row=0, column=0)
    episode_name_label = tk.Label(text_boxes_frame, text="Episode Name:")
    episode_name_label.grid(row=1, column=0)
    synopsis_label = tk.Label(text_boxes_frame, text="Synopsis:")
    synopsis_label.grid(row=2, column=0)

    # Create the text boxes
    part_id_entry = tk.Entry(text_boxes_frame)
    part_id_entry.grid(row=0, column=1)
    episode_name_entry = tk.Entry(text_boxes_frame)
    episode_name_entry.grid(row=1, column=1)
    synopsis_entry = tk.Entry(text_boxes_frame)
    synopsis_entry.grid(row=2, column=1)
    

    # Create the "Add" button
    add_button = tk.Button(admin_episodes_window, text="Add episode", command=lambda: add_episode(tree, part_id_entry, episode_name_entry, synopsis_entry))
    add_button.pack()

    # Create the label for displaying status
    status_label = tk.Label(admin_episodes_window, text="")
    status_label.pack()

    #---------------------------------------------------------------------------------------------------------------------------------------------------------------
    #--------------------------------------------------------------------------------------------------------------------------------------------------------
def open_admin_elements_window():
    admin_elements_window = tk.Toplevel(window)
    admin_elements_window.title("Admin Elements")

    # Create the tree view
    tree = ttk.Treeview(admin_elements_window, columns=("ElementID", "ElementReferenceName", "ElementDescription", "ElementType"))
    tree.heading("ElementID", text="Element ID")
    tree.heading("ElementReferenceName", text="Reference Name")
    tree.heading("ElementDescription", text="Description")
    tree.heading("ElementType", text="Element Type")
    tree.pack()

    # Create frames for interactive text boxes and labels
    text_boxes_frame = tk.Frame(admin_elements_window)
    text_boxes_frame.pack(pady=20)

    # Create the labels for text boxes
    element_ref_label = tk.Label(text_boxes_frame, text="ElementReferenceName:")
    element_ref_label.grid(row=0, column=0)
    element_desc_label = tk.Label(text_boxes_frame, text="ElementDescription:")
    element_desc_label.grid(row=1, column=0)
    element_type_label = tk.Label(text_boxes_frame, text="ElementType:")
    element_type_label.grid(row=2, column=0)

    # Create the text boxes
    element_ref_entry = tk.Entry(text_boxes_frame)
    element_ref_entry.grid(row=0, column=1)
    element_desc_entry = tk.Entry(text_boxes_frame)
    element_desc_entry.grid(row=1, column=1)
    options = [
        "Character",
        "Group",
        "Location",
        "Establishment",
        "Physical Collection",
        "Physical Object",
        "Idea",
        "Event",
        "Theme",
        "Easter Egg",
    ]

    # create combobox
    element_type_combobox = ttk.Combobox(text_boxes_frame, values=options)
    element_type_combobox.grid(row=2, column=1)

    # Create the "Add" button
    add_button = tk.Button(admin_elements_window, text="Add element", command=lambda: add_element(tree, element_ref_entry, element_desc_entry, element_type_combobox))
    add_button.pack()

    # Create the "Element Traits" button
    element_traits_button = tk.Button(admin_elements_window, text="Element Traits", command=lambda: show_element_traits(tree))
    element_traits_button.pack()

    # Create the label for displaying status
    status_label = tk.Label(admin_elements_window, text="")
    status_label.pack()

    def add_element(tree, element_ref_entry, element_desc_entry, element_type_combobox):
        # Get the values from text boxes
        element_reference_name = element_ref_entry.get()
        element_description = element_desc_entry.get()
        element_type_id = element_type_combobox.get()

        try:
            # Establish connection between Python and MySQL database via connector API
            connection = connector.connect(
                host="localhost",
                user="DisenchantmentAdmin",  # use your own
                password=password_entry.get(),  # use the entered password
                database="testdisenchantment",  # set the default schema
                raise_on_warnings=True
            )

            # Create a cursor object to execute SQL statements
            cursor = connection.cursor()

            # Check if the element_reference_name already exists
            cursor.execute("SELECT COUNT(*) FROM element WHERE ElementReferenceName=%s", (element_reference_name,))
            result = cursor.fetchone()

            if result[0] == 0:
                # Call the AddElement procedure with the provided values
                cursor.callproc("AddElement", (element_type_id, element_reference_name, element_description))
                connection.commit()
                status_label.config(text="Element added successfully.", fg="green")
                element_ref_entry.delete(0, tk.END)
                element_desc_entry.delete(0, tk.END)
                element_type_combobox.delete(0, tk.END)
            else:
                status_label.config(text="Element already exists.", fg="red")

            # Fetch data from the 'element' table
            cursor.execute("SELECT ElementID, ElementReferenceName, ElementDescription, ElementType FROM element")

            # Insert the data into the tree view
            tree.delete(*tree.get_children())

            for row in cursor:
                tree.insert("", tk.END, values=row)

        except connector.Error as err:
            print(f"Failed to add element: {err}")
            status_label.config(text="Failed to add element.", fg="red")

        finally:
            # Close the cursor and the connection
            cursor.close()
            connection.close()

    def show_element_traits(tree):
        selected_item = tree.focus()
        if selected_item:
            selected_element_ID = tree.item(selected_item)["values"][0]

            element_traits_window = tk.Toplevel(admin_elements_window)
            element_traits_window.title("Element Traits")

            # Create the tree view for element traits
            element_traits_tree = ttk.Treeview(element_traits_window, columns=("ElementReferenceName", "TraitName"))
            element_traits_tree.heading("ElementReferenceName", text="Element Reference Name")
            element_traits_tree.heading("TraitName", text="Trait Name")
            element_traits_tree.pack()

            try:
                # Establish connection between Python and MySQL database via connector API
                connection, cursor = quick_connect()

                # Fetch data from the 'addelementhastraitview' view
                cursor.execute(f"SELECT ElementReferenceName, TraitName FROM addelementhastraitview WHERE ElementID={selected_element_ID}")

                # Insert the data into the tree view
                for row in cursor:
                    element_traits_tree.insert("", tk.END, values=row)

            except connector.Error as err:
                print(f"Error fetching element traits: {err}")

            finally:
                # Close the cursor and the connection
                cursor.close()
                connection.close()

    # Establish connection between Python and MySQL database via connector API
    connection = connector.connect(
        host="localhost",
        user="DisenchantmentAdmin",  # use your own
        password=password_entry.get(),  # use the entered password
        database="testdisenchantment",  # set the default schema
        raise_on_warnings=True
    )

    try:
        # Create a cursor object to execute SQL statements
        cursor = connection.cursor()

        # Fetch data from the 'element' table
        cursor.execute("SELECT ElementID, ElementReferenceName, ElementDescription, ElementType FROM element")

        # Insert the data into the tree view
        tree.delete(*tree.get_children())

        for row in cursor:
            tree.insert("", tk.END, values=row)

    except connector.Error as err:
        print(f"Error fetching data from the 'element' table: {err}")

    finally:
        # Close the cursor and the connection
        cursor.close()
        connection.close()
#----------------------------------------------------------------------------------------------------------
#----------------------------------------------------------------------------------------------------------
def open_admin_scenes_window(selected_episode):
    admin_scenes_window = tk.Toplevel(window)
    admin_scenes_window.title(f"Episode {selected_episode} Scenes")

    # Create the tree view
    tree = ttk.Treeview(admin_scenes_window, columns=("SceneID", "SceneName", "SceneDescription", "EpisodeID", "Timestamp"))
    tree.heading("SceneID", text="Scene ID")
    tree.heading("SceneName", text="Scene Name")
    tree.heading("SceneDescription", text="Description")
    tree.heading("EpisodeID", text="Episode ID")
    tree.heading("Timestamp", text="Timestamp")
    tree.pack()

    # Fetch data from the 'scene' table
    try:
        # Establish connection between Python and MySQL database via connector API
        connection = connector.connect(
            host="localhost",
            user="DisenchantmentAdmin",  # use your own
            password=password_entry.get(),  # use the entered password
            database="testdisenchantment",  # set the default schema
            raise_on_warnings=True
        )

        # Create a cursor object to execute SQL statements
        cursor = connection.cursor()

        # Fetch data from the 'scene' table
        cursor.execute(f"SELECT SceneID, SceneName, SceneDescription, EpisodeID, Timestamp FROM scene WHERE EpisodeID = {selected_episode}")


        # Insert the data into the tree view
        for row in cursor:
            tree.insert("", tk.END, values=row)

    except connector.Error as err:
        print(f"Error fetching data from the 'scene' table: {err}")

    finally:
        # Close the cursor and the connection
        cursor.close()
        connection.close()

    # Create frames for interactive text boxes and labels
    text_boxes_frame = tk.Frame(admin_scenes_window)
    text_boxes_frame.pack(pady=20)

    # Create the labels for text boxes
    scene_name_label = tk.Label(text_boxes_frame, text="Scene Name:")
    scene_name_label.grid(row=0, column=0)
    scene_desc_label = tk.Label(text_boxes_frame, text="Scene Description:")
    scene_desc_label.grid(row=1, column=0)
    timestamp_label = tk.Label(text_boxes_frame, text="Timestamp:")
    timestamp_label.grid(row=2, column=0)

    # Create the text boxes
    scene_name_entry = tk.Entry(text_boxes_frame)
    scene_name_entry.grid(row=0, column=1)
    scene_desc_entry = tk.Entry(text_boxes_frame)
    scene_desc_entry.grid(row=1, column=1)
    timestamp_entry = tk.Entry(text_boxes_frame)
    timestamp_entry.grid(row=2, column=1)

    # Create the "Add" button
    add_button = tk.Button(admin_scenes_window, text="Add scene", command=lambda: add_scene(tree, scene_name_entry, scene_desc_entry, selected_episode, timestamp_entry))
    add_button.pack()


    # Create the "Open Scene" button
    open_scene_button = tk.Button(admin_scenes_window, text="Open Scene", command=lambda: open_scene_elements(admin_scenes_window, tree, cursor))
    open_scene_button.pack()

    # Create the label for displaying status
    status_label = tk.Label(admin_scenes_window, text="")
    status_label.pack()
##-----------------------------------------------------------------------------------------------------------------------------------------------
##-----------------------------------------------------------------------------------------------------------------------------------------------
    def add_scene(tree, scene_name_entry, scene_desc_entry, selected_episode, timestamp_entry):
        # Get the values from text boxes
        scene_name = scene_name_entry.get()
        scene_desc = scene_desc_entry.get()
        episode_ID = selected_episode
        timestamp = timestamp_entry.get()

        try:
            # Establish connection between Python and MySQL database via connector API
            connection = connector.connect(
                host="localhost",
                user="DisenchantmentAdmin",  # use your own
                password=password_entry.get(),  # use the entered password
                database="testdisenchantment",  # set the default schema
                raise_on_warnings=True
            )

            # Create a cursor object to execute SQL statements
            cursor = connection.cursor()

            # Check if the scene_reference_name already exists
            cursor.execute("SELECT COUNT(*) FROM scene WHERE SceneName=%s", (scene_name,))
            result = cursor.fetchone()

            if result[0] == 0:
                # Call the AddScene procedure with the provided values
                cursor.callproc("AddScene", (scene_name, episode_ID, scene_desc, timestamp))
                connection.commit()
                print("Scene added successfully.")
                status_label.config(text="Scene added successfully.", fg="green")
                scene_name_entry.delete(0, tk.END)
                scene_desc_entry.delete(0, tk.END)
                timestamp_entry.delete(0, tk.END)
            else:
                print("Scene name already exists.")
                status_label.config(text="Scene name already exists.", fg="red")

            # Fetch data from the 'scene' table
            cursor.execute("SELECT SceneID, SceneName, SceneDescription, EpisodeID, Timestamp FROM scene")

            # Insert the data into the tree view
            tree.delete(*tree.get_children())

            for row in cursor:
                tree.insert("", tk.END, values=row)

        except connector.Error as err:
            print(f"Failed to add scene: {err}")
            status_label.config(text="Failed to add scene.", fg="red")

        finally:
            # Close the cursor and the connection
            cursor.close()
            connection.close()
#-----------------------------------------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------------------------------------
##-----------------------------------------------------------------------------------------------------------------------------
##-----------------------------------------------------------------------------------------------------------------------------

def open_scene_elements(admin_scenes_window, tree, cursor):
    # Establish connection between Python and MySQL database via connector API
    connection, cursor = quick_connect()

    try:
        # Get the selected item
        selected_item = tree.focus()
        # Get the SceneName associated with the selected item
        scene_name = tree.item(selected_item)['values'][1]
        scene_ID = tree.item(selected_item)['values'][0]
        # Create a new window
        scene_elements_window = tk.Toplevel(admin_scenes_window)
        scene_elements_window.title(f"Admin {scene_name} includes elements")

        # Create a frame for the buttons
        scene_elements_frame = tk.Frame(scene_elements_window)
        scene_elements_frame.pack()

        # Create the spreadsheet (treeview) in the new window
        spreadsheet = ttk.Treeview(scene_elements_frame)
        spreadsheet.pack()

        # Add columns to the spreadsheet
        spreadsheet['columns'] = ('ElementReferenceName', 'InclusionDescription', 'TableName', 'SceneIncludesElementID','ElementID','SequentialID')
        spreadsheet.heading('#0', text='Scene Includes Elements')
        spreadsheet.heading('SceneIncludesElementID', text='Scene Includes Element ID')
        spreadsheet.heading('ElementReferenceName', text='Element Reference Name')
        spreadsheet.heading('InclusionDescription', text='Inclusion Description')
        spreadsheet.heading('TableName', text='Table Name')
        spreadsheet.heading('ElementID', text = 'Element ID')
        spreadsheet.heading('SequentialID', text = "SequentialID")
        

        # Create a cursor object to execute SQL statements
        cursor = connection.cursor()

        # Get the SceneID associated with the selected item
        scene_id = tree.item(selected_item)['values'][0]

        # Query the sceneview view to get the elements for the selected SceneID
        query = f"SELECT ElementReferenceName, InclusionDescription, TableName, SceneIncludesElementID, ElementID, SequentialID FROM sceneview WHERE sceneview.SceneID = {scene_id}"
        cursor.execute(query)
        elements = cursor.fetchall()

        # Insert elements into the spreadsheet
        for element in elements:
            spreadsheet.insert('', 'end', values=element)
        # Define the open_scene_includes_element function outside of open_scene_elements
        
        def open_scene_includes_element(scene_id):
            # Get the selected row from the spreadsheet
            connection, cursor = quick_connect()
            selected_row = spreadsheet.focus()
            ss_values = spreadsheet.item(selected_row)['values']
            # Create a new window for selecting the table name
            
            if ss_values:
                selected_table_name = ss_values[2]  # Get the TableName from the selected row
                selected_scene_includes_element_ID = ss_values[3] 
                selected_element_ID = ss_values[4]
                selected_element_desc = ss_values[1]
                selected_element_reference_name = ss_values[0]

                # Check if the selected table name is "other notes"
                if selected_table_name == "other notes":
                    inclusion_type_window = tk.Toplevel(scene_elements_window)
                    inclusion_type_window.title("Select Table Name")
                    # Create a label for the combo box
                    inclusion_type_label = tk.Label(inclusion_type_window, text="Table Name")
                    inclusion_type_label.pack()

                    # Fetch the available table names
                    available_table_names = [
                        "other notes",
                        "scene addresses element has trait",
                        "scene addresses element relationship",
                        "scene has example of element name",
                        "scene includes element has wish"
                    ]

                    # Create a combo box for selecting table name
                    table_name_combobox = ttk.Combobox(inclusion_type_window, values=available_table_names, width=30, state="readonly")
                    table_name_combobox.set(selected_table_name)
                    table_name_combobox.pack()
        

                    def create_scene_addresses_element_has_trait_window(selected_element_ID, selected_scene_includes_element_ID):
                        
                        # Create a new window for "scene addresses element has trait"
                        saeht_window = tk.Toplevel(inclusion_type_window)
                        saeht_window.title("Scene Addresses Element Has Trait")

                        # Call quick_connect to establish the connection and cursor
                        connection, cursor = quick_connect()
                        # Fetch the TraitName values
                        cursor.execute(f"SELECT TraitName FROM `trait`")
                        trait_names = [row[0] for row in cursor.fetchall()]

                        # Create a label for Trait
                        trait_label = tk.Label(saeht_window, text="Trait: ")
                        trait_label.pack()

                        # Create a combo box for selecting element traits
                        element_trait_combobox = ttk.Combobox(saeht_window, values=trait_names, width=127)
                        element_trait_combobox.pack()

                        # Create a label for the radio buttons
                        radio_label = tk.Label(saeht_window, text="Addressing Duration:")
                        radio_label.pack()

                        # Create the radio buttons
                        var = tk.IntVar()

                        R1 = tk.Radiobutton(saeht_window, text="Scene Includes Element is start of trait duration", variable=var, value=1)
                        R1.pack(anchor=tk.W)

                        R2 = tk.Radiobutton(saeht_window, text="Scene Includes Element is end of trait duration", variable=var, value=2)
                        R2.pack(anchor=tk.W)

                        R3 = tk.Radiobutton(saeht_window, text="Scene Includes Element Includes trait", variable=var, value=3)
                        R3.pack(anchor=tk.W)

                        # Create label textbox
                        scene_addresses_element_has_trait_description_label = ttk.Label(saeht_window, text ="How does this scene address the element having the trait?")
                        scene_addresses_element_has_trait_description_label.pack()

                        # Create description textbox
                        scene_addresses_element_has_trait_description_textbox = ttk.Entry(saeht_window, width=127)
                        scene_addresses_element_has_trait_description_textbox.pack()

                        # Create instance label and textbox
                        scene_addresses_element_trait_instance_label = ttk.Label(saeht_window, text ="If not the only instance of this element relationship, indicate which instace this refers to.")
                        scene_addresses_element_trait_instance_label.pack()

                        scene_addresses_element_trait_instance_textbox = ttk.Entry(saeht_window, width=5)
                        scene_addresses_element_trait_instance_textbox.pack()

                        # Define the command for the "Add" button here
                        def add_button_command(selected_element_ID, selected_scene_includes_element_ID):
                            instance = scene_addresses_element_trait_instance_textbox.get()
                            scene_addresses_element_has_trait_description = scene_addresses_element_has_trait_description_textbox.get()
                            # Connect to the database
                            connection, cursor = quick_connect()
                            try:
                                # Check if there is a matching TraitName
                                cursor.execute(f"SELECT TraitID FROM `trait` WHERE LOWER(TraitName) = LOWER('{element_trait_combobox.get()}')")
                                trait_row = cursor.fetchone()
                                if trait_row:
                                    combobox_trait_id = trait_row[0]
                                    print(combobox_trait_id)
                                else:
                                    # Call AddTrait procedure
                                    try:
                                        cursor.callproc("AddTrait", (element_trait_combobox.get(), None))
                                        connection.commit()
                                    except connector.Error as error:
                                        print(f"Error accessing scene elements: {error}")
                                        
                                    # Check if there is a matching TraitName again
                                    cursor.execute(f"SELECT TraitID FROM `trait` WHERE LOWER(TraitName) = LOWER('{element_trait_combobox.get()}')")
                                    trait_row = cursor.fetchone()
                                    if trait_row:
                                        combobox_trait_id = trait_row[0]
                                    else:
                                        print("Failed to add trait")

                                # Check if there is a matching row in elementhastrait
                                cursor.execute(f"SELECT ElementHasTraitID, TraitDurationID FROM `elementhastrait` WHERE ElementID = {selected_element_ID} AND TraitID = {combobox_trait_id}")
                                element_trait_row = cursor.fetchone()
                                if element_trait_row:
                                    selected_element_has_trait_id = element_trait_row[0]
                                    trait_duration_id = element_trait_row[1]
                                else:

                                    # Call the AddNullTraitDuration procedure

                                    try:
                                        cursor.callproc("AddNullTraitDuration")
                                        connection.commit()
                                        select_query = "SELECT LAST_INSERT_ID()"
                                        cursor.execute(select_query)
                                        result = cursor.fetchall()
                                        trait_duration_id = result[0][0]
                                    except connector.Error as error:
                                        print(f"Error with NullTraitDuration procedure: {error}")

                                    # Call AddElementHasTrait procedure

                                    cursor.execute(f"CALL `AddElementHasTrait`({selected_element_ID}, {combobox_trait_id}, {trait_duration_id} , NULL, NULL, {instance})")
                                    connection.commit()
                                    # Check if there is a matching row in elementhastrait again
                                    cursor.execute(f"SELECT ElementHasTraitID FROM `elementhastrait` WHERE ElementID = {selected_element_ID} AND TraitID = {combobox_trait_id}")
                                    element_trait_row = cursor.fetchone()
                                if element_trait_row:
                                    selected_element_has_trait_id = element_trait_row[0]
                            
                                else:
                                    print("Failed to add element has trait")

                                if var.get() == 1:
                                    cursor.execute(f"SELECT TraitDurationID FROM `elementhastrait` WHERE ElementHasTraitID = {selected_element_has_trait_id}")
                                    trait_duration_row = cursor.fetchone()
                                    if trait_duration_row:
                                        selected_trait_duration_id = trait_duration_row[0]
                                    try:
                                        cursor.execute(f"CALL ModifyTraitDuration({selected_trait_duration_id},'{scene_addresses_element_has_trait_description_textbox.get()}', NULL, {selected_scene_includes_element_ID}, NULL)")
                                        connection.commit()
                                    except connector.Error as error:
                                        print(f"Error with ModifyTraitDuration procedure: {error}")
                                        
                                if var.get() == 2:
                                    cursor.execute(f"SELECT TraitDurationID FROM `elementhastrait` WHERE ElementHasTraitID = {selected_element_has_trait_id}")
                                    trait_duration_row = cursor.fetchone()
                                    if trait_duration_row:
                                        selected_trait_duration_id = trait_duration_row[0]
                                    try:
                                        cursor.execute(f"CALL ModifyTraitDuration({selected_trait_duration_id}, NULL, '{scene_addresses_element_has_trait_description_textbox.get()}', NULL, {selected_scene_includes_element_ID})")
                                        connection.commit()
                                    except connector.Error as error:
                                        print(f"Error with ModifyTraitDuration procedure: {error}")
                                
                                
                                #AddSceneAddressesElementHasTrait
                                try:
                                    cursor.callproc("AddSceneAddressesElementHasTrait", (selected_element_has_trait_id, scene_addresses_element_has_trait_description, selected_scene_includes_element_ID))
                                    connection.commit()
                                    scene_elements_window.destroy()
                                    open_scene_elements(admin_scenes_window, tree, cursor)
                                except connector.Error as error:
                                    print(f"Error with AddSceneAddressesElementHasTriat: {error}")

                            finally:
                                # Close the database connection
                                quick_close()
                        for item in spreadsheet.get_children():
                            spreadsheet.delete(item)

                        # Add button
                        add_button = tk.Button(saeht_window, text="add scene addresses element has trait", command= lambda: add_button_command(selected_element_ID, selected_scene_includes_element_ID))
                        add_button.pack()
                    def create_scene_has_example_of_element_name_window(selected_element_ID, selected_scene_includes_element_ID):
                        
                        # Create a new window for "scene has example of element name"
                        sheen_window = tk.Toplevel(inclusion_type_window)
                        sheen_window.title("Scene has example of element name")

                        # Call quick_connect to establish the connection and cursor
                        connection, cursor = quick_connect()
                        # Fetch the ElementName values based on selected_element_ID
                        cursor.execute(f"SELECT ElementName FROM `addelementnameview` WHERE {selected_element_ID} = addelementnameview.ElementID")
                        element_names = [row[0] for row in cursor.fetchall()]

                        # Create a label for Element Name
                        element_name_label = tk.Label(sheen_window, text="Name: ")
                        element_name_label.pack()

                        # Create a combo box for selecting element traits
                        element_name_combobox = ttk.Combobox(sheen_window, values=element_names, width=127)
                        element_name_combobox.pack()

                        # Create label textbox
                        scene_addresses_element_has_name_description_label = ttk.Label(sheen_window, text ="How does this scene address the element having the name?")
                        scene_addresses_element_has_name_description_label.pack()

                        # Create description textbox
                        scene_addresses_element_has_name_description_textbox = ttk.Entry(sheen_window, width=127)
                        scene_addresses_element_has_name_description_textbox.pack()

                        # Define the command for the "Add" button here
                        def add_scene_addresses_element_has_name_command(selected_element_ID, selected_scene_includes_element_ID):
                            scene_addresses_element_has_name_description = scene_addresses_element_has_name_description_textbox.get()
                            # Connect to the database
                            connection, cursor = quick_connect()
                            try:
                                # Check if there is a matching ElementName
                                element_name = element_name_combobox.get()
                                cursor.execute(f"SELECT ElementNameID FROM `elementname` WHERE LOWER(ElementName) = LOWER('{element_name}')")
                                name_row = cursor.fetchone()

                                if name_row:
                                    name_id = name_row[0]
                                else:
                                    # Call AddElementName procedure
                                    try:
                                        cursor.callproc("AddElementName", (element_name, None, None))
                                        connection.commit()
                                    except connector.Error as error:
                                        print(f"Error accessing scene elements: {error}")
                                        return

                                    # Check if there is a matching ElementName again
                                    cursor.execute(f"SELECT ElementNameID FROM `elementname` WHERE LOWER(ElementName) = LOWER('{element_name}')")
                                    name_row = cursor.fetchone()

                                    if name_row:
                                        name_id = name_row[0]
                                    else:
                                        print("Failed to add name")
                                        return


                                # Check if there is a matching row in elementhaselementname
                                cursor.execute(f"SELECT ElementHasNameID FROM `elementhaselementname` WHERE ElementID = {selected_element_ID} AND ElementNameID = {name_id}")
                                element_name_row = cursor.fetchone()
                                if element_name_row:
                                    selected_element_has_name_id = element_name_row[0]
                                    # Call AddElementHasElementName procedure
                                else:
                                    cursor.execute(f"CALL `AddElementHasElementName`({selected_element_ID}, {name_id})")
                                    connection.commit()
                                    # Check if there is a matching row in elementhaselementname again
                                    cursor.execute(f"SELECT ElementHasNameID FROM `elementhaselementname` WHERE ElementID = {selected_element_ID} AND ElementNameID = {name_id}")
                                    element_name_row = cursor.fetchone()
                                if element_name_row:
                                    selected_element_has_name_id = element_name_row[0]
                            
                                else:
                                    print("Failed to add element has name")
                        
                                #AddSceneHasExampleOfElementHasElementName
                                try:
                                    cursor.callproc("AddSceneHasExampleOfElementName", (selected_element_has_name_id, scene_addresses_element_has_name_description, selected_scene_includes_element_ID))
                                    connection.commit()
                                    scene_elements_window.destroy()
                                    open_scene_elements(admin_scenes_window, tree, cursor)
                                except connector.Error as error:
                                    print(f"Error with AddSceneHasExampleOfElementName: {error}")
                            
                            except UnboundLocalError as error:
                                print(f"Error: {error}")

                            finally:
                                # Close the database connection
                                quick_close()
                        for item in spreadsheet.get_children():
                            spreadsheet.delete(item)

                        # Add button
                        add_button = tk.Button(sheen_window, text="add scene addresses element has name", command= lambda: add_scene_addresses_element_has_name_command(selected_element_ID, selected_scene_includes_element_ID))
                        add_button.pack()
####----------------------------------------------------------------------------------------------------------------------------------
###----------------------------------------------------------------------------------------------------------------------------------
#---------------------------------------------------------------------------------------------------------------------------
                    def create_scene_addresses_element_relationship(selected_element_ID, selected_scene_includes_element_ID,selected_element_reference_name):
                            
                        # Create a new window for "scene has example of element relationship"
                        saer_window = tk.Toplevel(inclusion_type_window)
                        saer_window.title("Scene has example of element relationship")

                        # Call quick_connect to establish the connection and cursor
                        connection, cursor = quick_connect()

                        # Fetch the existing relationships values based on selected_element_ID
                        cursor.execute(f"SELECT RelationshipID, RelationshipType, Element2ID, ElementReferenceName, RelationshipDurationID FROM `addelementrelationshipview` WHERE {selected_element_ID} = addelementrelationshipview.Element1ID")
                        e1relationships = [(row[0], row[1], row[2], row[3], row[4]) for row in cursor.fetchall()]
                        print(e1relationships)
                        e1rtoe2s = list((row[1], row[2]) for row in e1relationships)
                        print(e1rtoe2s)
                        element2IDs = list(set(row[1] for row in e1rtoe2s))

                        if element2IDs != [None]:  # This will check if the list is not empty
                            #clear all none values from element2IDs
                            element2IDs = [id for id in element2IDs if id is not None]
                            element2IDs_str = ', '.join(str(id) for id in element2IDs)
                            query = f"SELECT ElementReferenceName FROM element WHERE ElementID IN ({element2IDs_str})"
                            cursor.execute(query)
                            # Fetch the results
                            element2_reference_names = [row[0] for row in cursor.fetchall()]
                        else:
                            element2_reference_names = []

                        def update_relationshiptype_values(*args):
                            selected_element_2 = element_2_combobox.get()
                            
                            # Filter the e1rtoe2s list based on the selected element_2
                            filtered_e1rtoe2s = [row[2] for row in e1rtoe2s if row[1] == selected_element_2]
                            
                            # Update the values in the relationshiptype_combobox
                            relationshiptype_combobox['values'] = filtered_e1rtoe2s

                                               
                        # Create a label for Element Name
                        element1_name_label = tk.Label(saer_window, text=f"{selected_element_reference_name} Has relationship to:")
                        element1_name_label.pack()

                        # Create a combo box for selecting element 2
                        element_2_combobox = ttk.Combobox(saer_window, values=element2_reference_names, width=127)
                        element_2_combobox.pack()

                        # Bind the update_relationshiptype_values function to the <<ComboboxSelected>> event of element_2_combobox
                        element_2_combobox.bind("<<ComboboxSelected>>", update_relationshiptype_values)


                        # Create a label for relationship type
                        relationship_type_label = tk.Label(saer_window, text=f"Relationship of {selected_element_reference_name} to element 2:")
                        relationship_type_label.pack()

                        # Create a combo box for the relationshipType 

                        relationshiptype_combobox = ttk.Combobox(saer_window, values=[], width=127)
                        relationshiptype_combobox.pack()

                        # Create description label
                        scene_addresses_element_relationship_description_label = ttk.Label(saer_window, text ="How does this scene address the element relationship to the 2nd element?")
                        scene_addresses_element_relationship_description_label.pack()

                        # Create description textbox
                        scene_addresses_element_relationship_description_textbox = ttk.Entry(saer_window, width=127)
                        scene_addresses_element_relationship_description_textbox.pack()

                        # Create instance label textbox
                        
                        scene_addresses_element_relationship_instance_label = ttk.Label(saer_window, text ="If not the only instance of this element relationship, indicate which instace this refers to.")
                        scene_addresses_element_relationship_instance_label.pack()

                        scene_addresses_element_relationship_instance_textbox = ttk.Entry(saer_window, width=5)
                        scene_addresses_element_relationship_instance_textbox.pack()

                        ## Create a inheritance checkbox
                        inherits_var = tk.BooleanVar()
                        inherits_var.set(False)  # Set the initial value to 0

                        checkbox = tk.Checkbutton(saer_window, text="Element 1 inherits traits from Element 2", variable=inherits_var, onvalue=True, offvalue=False)
                        
                        # Create a label for the radio buttons
                        radio_label = tk.Label(saer_window, text="Addressing Duration:")
                        radio_label.pack()

                        # Create the radio buttons
                        var = tk.IntVar()

                        R1 = tk.Radiobutton(saer_window, text="Scene Includes Element is start of relationship duration", variable=var, value=1)
                        R1.pack(anchor=tk.W)

                        R2 = tk.Radiobutton(saer_window, text="Scene Includes Element is end of relationship duration", variable=var, value=2)
                        R2.pack(anchor=tk.W)

                        R3 = tk.Radiobutton(saer_window, text="Scene Includes Element relationship", variable=var, value=3)
                        R3.pack(anchor=tk.W)


                        # Define the command for the "Add" button here
                        def add_scene_addresses_element_relationship_command(scene_addresses_element_relationship_description_textbox, selected_element_ID, selected_scene_includes_element_ID, element_2_combobox, relationshiptype_combobox, inherits_var):
                            # Retrieve the values from the widgets
                            scene_addresses_element_relationship_description = scene_addresses_element_relationship_description_textbox.get()
                            element2name = element_2_combobox.get()
                            relationshiptype = relationshiptype_combobox.get()
                            inherits = inherits_var
                            instance = scene_addresses_element_relationship_instance_textbox.get()
                            connection, cursor = quick_connect()
                            #check to see if there is a value for the element2
                            
                            cursor.execute(f"SELECT ElementID FROM `element` WHERE LOWER(ElementReferenceName) = LOWER('{element2name}')")
                            id_row = cursor.fetchone()
                            if id_row:
                                element2_id = int(id_row[0])
                                try:
                                    # Check if there is a matching relationship
                                    cursor.execute(f"SELECT ElementHasRelationshipToElementID, RelationshipID FROM `addelementrelationshipview` WHERE Element1ID = '{selected_element_ID}' AND Element2ID = '{element2_id}' AND RelationshipType = '{relationshiptype}'AND RelationshipInstance = '{instance}'")
                                    relationship_row = cursor.fetchone()
                                    if relationship_row:
                                        element_has_relationship_id = relationship_row[0]
                                        relationship_id = relationship_row[1]
                                    else:
                                        # Check if there is a matching relationship type
                                        cursor.execute(f"SELECT RelationshipID FROM `relationship` WHERE RelationshipType = '{relationshiptype}'")
                                        relationship_type_row = cursor.fetchone()
                                        if not relationship_type_row:
                                            try:                                            
                                                cursor.callproc("AddRelationship", (relationshiptype, None))
                                                connection.commit()
                                                select_query = "SELECT LAST_INSERT_ID()"
                                                cursor.execute(select_query)
                                                result = cursor.fetchall()
                                                relationship_id = result[0][0]
                                            except connector.Error as error:
                                                print(f"Error adding relationship with relationship type: {error}")
                                                return
                                        
                                        
                                        # Call add null relationship duration procedure
                                        try:
                                            cursor.callproc("AddNullRelationshipDuration")
                                            connection.commit()
                                            select_query = "SELECT LAST_INSERT_ID()"
                                            cursor.execute(select_query)
                                            result = cursor.fetchall()
                                            relationship_duration_id = result[0][0]
                                        except connector.Error as error:
                                            print(f"Error with NullRelationshipDuration procedure: {error}")
                                        # Call addelementhasrelationship procedure

                                        try:
                                            cursor.callproc("AddElementHasRelationship", (selected_element_ID, relationship_id, element2_id, None, relationship_duration_id, None, instance))
                                            connection.commit()
                                            select_query = "SELECT LAST_INSERT_ID()"
                                            cursor.execute(select_query)
                                            result = cursor.fetchall()
                                            element_has_relationship_id = result[0][0]
                                        except connector.Error as error:
                                            print(f"Error with AddElementHasRelationshipDuration procedure: {error}")
                                    

                                        # Check for the  matching relationship again

                                        cursor.execute(f"SELECT RelationshipID FROM `relationship` WHERE RelationshipType ='{relationshiptype}'")
                                        type_results = cursor.fetchall()
                                        cursor.execute(f"SELECT RelationshipID FROM `elementhasrelationshiptoelement` WHERE Element1ID = {selected_element_ID} AND Element2ID = {element2_id}")
                                        relationship_rows = cursor.fetchall()

                                        if len(list(set(relationship_rows) & set(type_results)))>= 1:
                                            relationship_id = list(set(relationship_rows) & set(type_results))[0][0]
                                        else:
                                            print("Failed to add relationship")
                                        _ = cursor.fetchall()

                            
                                    #AddSceneAddressesElementRelationship
                                    try:
                                        cursor.callproc("AddSceneAddressesElementRelationship", (element_has_relationship_id, scene_addresses_element_relationship_description, selected_scene_includes_element_ID, selected_element_ID, relationship_id))
                                        connection.commit()
                                        scene_elements_window.destroy()
                                        open_scene_elements(admin_scenes_window, tree, cursor)
                                    except connector.Error as error:
                                        print(f"Error with AddSceneHasExampleOfElementRelationship: {error}")
                                    
                                    #set relationship duration if needed
                                    if var.get() == 1:
                                        cursor.execute(f"SELECT RelationshipDurationID FROM `elementhasrelationshiptoelement` WHERE ElementHasRelationshipToElementID = {element_has_relationship_id}")
                                        relationship_duration_row = cursor.fetchall()
                                        cursor.close()
                                        cursor = connection.cursor()
                                        if relationship_duration_row:
                                            selected_relationship_duration_id = relationship_duration_row[0][0]
                                        try:
                                            cursor.execute(f"CALL ModifyRelationshipDuration({selected_relationship_duration_id},'{scene_addresses_element_relationship_description}', NULL, {selected_scene_includes_element_ID}, NULL)")
                                            connection.commit()
                                        except connector.Error as error:
                                            print(f"Error with ModifyRelationshipDuration procedure: {error}")
                                            
                                    if var.get() == 2:
                                        cursor.execute(f"SELECT RelationshipDurationID FROM `elementhasrelationshiptoelement` WHERE ElementHasRelationshipToElementID = {element_has_relationship_id}")
                                        relationship_duration_row = cursor.fetchall()
                                        cursor.close()
                                        cursor = connection.cursor()
                                        print("relationship duration row", relationship_duration_row)
                                        
                                        if relationship_duration_row:
                                            selected_relationship_duration_id = relationship_duration_row[0][0]
                                        try:
                                            cursor.execute(f"CALL ModifyRelationshipDuration({selected_relationship_duration_id},'{scene_addresses_element_relationship_description}', NULL, {selected_scene_includes_element_ID}, NULL)")
                                            connection.commit()
                                        except connector.Error as error:
                                            print(f"Error with ModifyRelationshipDuration procedure: {error}")
                                
                                except UnboundLocalError as error:
                                    print(f"Error: {error}")

                                finally:
                                    # Close the database connection
                                    quick_close()
                                    
                            else:
                                print("no element 2 with value: ", element2name)
                        

                        # Add button
                        add_button = tk.Button(saer_window, text="add scene addresses element relationship", command=lambda: add_scene_addresses_element_relationship_command(scene_addresses_element_relationship_description_textbox, selected_element_ID, selected_scene_includes_element_ID, element_2_combobox, relationshiptype_combobox, inherits_var))
                        # Define the variables with desired values
                        scene_addresses_element_relationship_description = scene_addresses_element_relationship_description_textbox.get()
                        element2name = element_2_combobox.get()
                        relationshiptype = relationshiptype_combobox.get()
                        inherits = inherits_var
                        add_button.pack()

##-------------------------------------------------------------------------------------------------------------------------------------
##----------------------------------inclusion type tree-------------------------------------------------------------
##------------------------------------------------------------------------------------------------------------------------
                    def define_scene_includes_element(selected_element_ID, selected_scene_includes_element_ID, selected_table_name, selected_element_reference_name):
                        if table_name_combobox.get() == "scene addresses element has trait":
                            create_scene_addresses_element_has_trait_window(selected_element_ID, selected_scene_includes_element_ID)
                        elif table_name_combobox.get() == "scene has example of element name":
                            create_scene_has_example_of_element_name_window(selected_element_ID, selected_scene_includes_element_ID)
                        elif table_name_combobox.get() == "scene addresses element relationship":
                            create_scene_addresses_element_relationship(selected_element_ID, selected_scene_includes_element_ID, selected_element_reference_name)
                        else:
                            print("Invalid selection")
                        refresh_spreadsheet()

                    # Create the "Define" button
                    define_button = tk.Button(inclusion_type_window, text="Define", command=lambda: define_scene_includes_element(selected_element_ID, selected_scene_includes_element_ID, selected_table_name, selected_element_reference_name))
                    define_button.pack()

        # Create the "refine" button
        open_button = tk.Button(scene_elements_frame, text="Refine other notes for scene includes element", command= lambda: open_scene_includes_element(scene_id))
        open_button.pack()

        
        # Fetch "ElementReferenceName" values from the "element" table
        cursor.execute("SELECT ElementReferenceName, ElementID FROM element")
        element_data = cursor.fetchall()
        element_names = [row[0] for row in element_data]
        element_ids = [row[1] for row in element_data]

        # Create a label for the combobox
        element_label = tk.Label(scene_elements_frame, text="Element")
        element_label.pack()

        # Create a combobox for selecting element
        element_combobox = ttk.Combobox(scene_elements_frame, values=element_names, state="readonly")
        element_combobox.pack()

        # Create a label for the description entry
        description_label = tk.Label(scene_elements_frame, text="Description")
        description_label.pack()

        # Create an entry for entering description
        description_entry = tk.Entry(scene_elements_frame)
        description_entry.pack()

        # Create a label for the sequential ID entry
        sequential_id_label = tk.Label(scene_elements_frame, text="Sequential ID")
        sequential_id_label.pack()

        # Create an entry for entering sequential ID
        sequential_id_entry = tk.Entry(scene_elements_frame)
        sequential_id_entry.pack()

        def add_scene_includes_element(scene_id):
            selected_element_index = element_combobox.current()
            if selected_element_index >= 0:
                new_element_id = element_ids[selected_element_index]
                new_description = description_entry.get()
                new_sequential_id = sequential_id_entry.get()

                # Establish a new connection within the function
                try:
                    connection_inner, cursor_inner = quick_connect()

                    # Call the AddSceneIncludesElement procedure
                    cursor_inner.callproc("AddSceneIncludesElement", (scene_id, new_element_id, new_description, new_sequential_id))
                    connection_inner.commit()

                    # Refresh the spreadsheet with updated data
                    spreadsheet.delete(*spreadsheet.get_children())
                    cursor_inner.execute(query)
                    updated_elements = cursor_inner.fetchall()
                    for element in updated_elements:
                        spreadsheet.insert('', 'end', values=element)

                    # Clear the entry fields
                    description_entry.delete(0, 'end')
                    sequential_id_entry.delete(0, 'end')

                finally:
                    # Close the inner connection
                    connection_inner.close()

        # Create a button to add scene includes element
        add_scene_includes_button = tk.Button(scene_elements_frame, text="Add scene includes element", command= lambda: add_scene_includes_element(scene_id))
        add_scene_includes_button.pack()
        
        def refresh_spreadsheet():
            # Clear the existing elements in the spreadsheet
            connection, cursor = quick_connect()
            for item in spreadsheet.get_children():
                spreadsheet.delete(item)
                
            # Query the sceneview view to get the updated elements for the selected SceneID
            query = f"SELECT ElementReferenceName, InclusionDescription, TableName, SceneIncludesElementID, ElementID, SequentialID FROM sceneview WHERE sceneview.SceneID = {scene_id}"
            cursor.execute(query)
            updated_elements = cursor.fetchall()

            # Insert the updated elements into the spreadsheet
            for element in updated_elements:
                spreadsheet.insert('', 'end', values=element)   
            quick_close()
    except connector.Error as error:
        print(f"Error accessing scene elements: {error}")
        # Close the connection if an error occurs
        connection.close()


    


#------------------------------------------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------------------------------------------
def open_admin_traits_window():
    admin_traits_window = tk.Toplevel(window)
    admin_traits_window.title("Admin traits")
        
        
    # Create the tree view
    tree = ttk.Treeview(admin_traits_window, columns=("TraitID", "TraitName", "TraitUniversalDescription", ))
    tree.heading("TraitID", text="Trait ID")
    tree.heading("TraitName", text="Trait")
    tree.heading("TraitUniversalDescription", text="Description")
    tree.pack()

    # Fetch data from the 'traits' table
    try:
        connection, cursor = quick_connect()

        # Fetch data from the 'episodes' table
        cursor.execute("SELECT TraitID, TraitName, TraitUniversalDescription FROM trait")

        # Insert the data into the tree view
        for row in cursor:
            tree.insert("", tk.END, values=row)

    except connector.Error as err:
        print(f"Error fetching data from the 'episodes' table: {err}")

    finally:
        # Close the cursor and the connection
        cursor.close()
        connection.close()

    # Create frames for interactive text boxes and labels
    text_boxes_frame = tk.Frame(admin_traits_window)
    text_boxes_frame.pack(pady=20)

    # Create the labels for text boxes
    trait_name_label = tk.Label(text_boxes_frame, text="Trait Name:")
    trait_name_label.grid(row=0, column=0)
    trait_universal_description_label = tk.Label(text_boxes_frame, text="Trait description:")
    trait_universal_description_label.grid(row=1, column=0)

    # Create the text boxes
    trait_name_entry = tk.Entry(text_boxes_frame)
    trait_name_entry.grid(row=0, column=1)
    trait_universal_description_entry = tk.Entry(text_boxes_frame)
    trait_universal_description_entry.grid(row=1, column=1)


    # Create the "Add" button
    add_button = tk.Button(admin_traits_window, text="Add trait", command=lambda: add_trait(tree, trait_name_entry, trait_universal_description_entry))
    add_button.pack()

    # Create the label for displaying status
    status_label = tk.Label(admin_traits_window, text="")
    status_label.pack()
#------------------------------------------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------------------------------------------


# Create the main window
window = tk.Tk()
window.title("Main Window")

# Create a frame for the password label and entry
password_frame = tk.Frame(window)
password_frame.pack(pady=20)

# Create the password label
password_label = tk.Label(password_frame, text="Password:")
password_label.pack(side=tk.LEFT)

# Create the password entry widget
password_entry = tk.Entry(password_frame, show="*")
password_entry.pack(side=tk.LEFT)

# Create the "Connect" button
connect_button = tk.Button(window, text="Connect", command= connect_to_database)
connect_button.pack()

# Create the "Close Connection" button
close_button = tk.Button(window, text="Close Connection", command= close_connection)
close_button.pack()

# Create the connection status label
connection_label = tk.Label(window, text="Not Connected")
connection_label.pack()
#----------------------------------------------------------------------------
#----------------------------------------------------------------------------
# Start the Tkinter event loop
window.mainloop()


Connection between MySQL and Python is established.

Connection to the database is closed.

